<a href="https://colab.research.google.com/github/abudubai16/CNN-using-transfer-learning/blob/main/Simple_RAG_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A Simple implementation of a LLM with RAG support

The work over here is based heavily on the YouTube video: https://www.youtube.com/watch?v=Ylz779Op9Pw -Shawn Talebi

## Downloading the required libraries

In [ ]:
!pip install llama-index -q -q
!pip install llama-index-embeddings-huggingface -q -q
!pip install peft -q -q
!pip install auto-gptq -q -q
!pip install optimum -q -q
!pip install bitsandbytes -q -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 83.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.2/853.2 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

## Importing the embedding model

In [ ]:
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

Settings.llm = None
Settings.chunk_size = 256
Settings.chunk_overlap = 25

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

LLM is explicitly disabled. Using MockLLM.


## Creating the vector embeddings

In [ ]:
pdf_file_path = '/content/'

documents = SimpleDirectoryReader(pdf_file_path).load_data()

In [ ]:
print(len(documents))
t = 3
for pos, doc in enumerate(documents):
  if pos == t:
    print(doc.text)
    break

33
51 Chapter 9: Rights  & Morals  
By Amit Golder  
 
I am not a philosopher, philosophy lecturer nor a particularly good/hard working philosophy 
student. Much of the content of this is stolen from  first year philosophy subjects, so apologies 
for boring some of you to death. With that caveat in mind, please enjoy this brief introduction 
to moral and rights theory. If you want to know more, use the words/names that are bold and 
underlined  as the star t of your wikipedia -ing/ actual research.  
 
1 MORALS  
 
Utility vs Deontology  – the central dichotomy of all moral discussions. Should we analyse 
ideas and conduct by looking to their consequences or their intrinsic moral rightness or 
wrongness?  
 
Utility  – something is good if it leads to the best outcomes  
- But what are the best outcomes? Most preferences fulfilled? Most urgent preferences 
fulfilled? Greatest net happiness?  
- No extra importance is placed on the lives of those with special relationships to you 
(fami

## Creating the vector storage and query engine

In [ ]:
index = VectorStoreIndex.from_documents(documents)

In [ ]:
top_k = 3
retriever = VectorIndexRetriever(index, similarity_top_k = top_k)

In [ ]:
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)],
)

In [ ]:
def get_context(qns:str, top_k: int):
  response = query_engine.query(qns)

  context = 'Context:\n'
  for i in range(top_k):
    context = context + response.source_nodes[i].text + '\n'

  return context

## Importing and configuring the LLM

In [ ]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
# To use a different branch, change revision
# For example: revision="gptq-4bit-32g-actorder_True"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4481: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.2-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.la

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

## Vector retrieval and basic prompting

In [ ]:
prompt_template_w_context = lambda context, comment: f"""DebateGPT, functioning as an Asian Parliamentary debate teacher on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. \
It reacts to feedback aptly and ends responses with its signature '–Debate Coach'. \
DebateGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, \
thus keeping the interaction natural and engaging.

{context}

Please respond to the following comment. Use the context above if it is helpful.
{comment}
"""

In [ ]:
def get_reply(prompt: str)->str:
  model.eval()

  inputs = tokenizer(prompt, return_tensors='pt')
  outputs = model.generate(input_ids=inputs['input_ids'].to('cuda'), max_new_tokens=280)

  output_str = tokenizer.batch_decode(outputs)[0].splitlines()[-1]
  return output_str

In [ ]:
comment = input("Enter your comment: ")
context = get_context(comment, top_k)
prompt  = prompt_template_w_context(context, comment)

reply = get_reply(prompt)
print(f'Generated Result: \n {reply}')

Enter your comment: is it ok to offend your opponent in a debate 


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated Result: 
 In the end, the utility of offending your opponent in a debate depends on the specific context and the goals of the debate. If the debate is solely focused on utility and the consequences of the arguments, then offending your opponent might be an effective strategy. However, if the debate also involves principles or moral considerations, then offending your opponent could potentially undermine your credibility and weaken your argument.
